In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import time

# URL base de la sección de política en Verifica EFE
BASE_URL = "https://verifica.efe.com/politica/page/"
total_pages = 60  # Número de páginas a recorrer


In [23]:

def initialize_driver():
    """
    Inicializa el driver de Selenium con configuración para ejecución en modo headless.
    """
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    service = Service('chromedriver/chrome.exe')  # Ruta al ejecutable de ChromeDriver
    return webdriver.Chrome(options=options)

# Inicializa el driver y herramientas auxiliares
driver = initialize_driver()
wait = WebDriverWait(driver, 10)
actions = ActionChains(driver)

# Listas para almacenar los datos recopilados
titles, dates, contents, urls = [], [], [], []

try:
    for page in range(1, total_pages + 1):
        # Navegar a la página actual
        driver.get(BASE_URL + str(page))
        time.sleep(5)  # Esperar a que la página cargue completamente

        # Procesar el HTML de la página con BeautifulSoup
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")

        # Encontrar todos los enlaces a las noticias en la página actual
        news_links = soup.find_all("a", class_="icons")

        for link in news_links:
            # Obtener el enlace de cada noticia
            article_url = link.get("href")
            urls.append(article_url)

            # Navegar a la noticia
            driver.get(article_url)
            time.sleep(3)  # Esperar a que la noticia cargue

            # Procesar el HTML de la noticia
            article_source = driver.page_source
            article_soup = BeautifulSoup(article_source, "html.parser")

            # Extraer el título de la noticia
            title = article_soup.find("h1", class_="sc_layouts_title_caption").get_text(strip=True)
            titles.append(title)

            # Extraer la fecha de la noticia
            date = article_soup.find("span", class_="post_meta_item post_date").get_text(strip=True)
            dates.append(date)

            # Extraer el contenido completo de la noticia
            paragraphs = article_soup.find_all("p")
            content = " ".join([p.get_text(strip=True) for p in paragraphs])
            contents.append(content)

            # Volver a la página anterior
            driver.back()
            time.sleep(2)

finally:
    driver.quit()

# Crear un DataFrame con los datos recopilados
news_data = pd.DataFrame({
    'Título': titles,
    'Fecha': dates,
    'Contenido': contents,
    'URL': urls
})

# Guardar los datos en un archivo CSV
news_data.to_csv("noticias_efe.csv", index=False)
print("Datos guardados en 'noticias_efe.csv'")
